In [1]:
from datetime import *
import pandas as pd
import glob
import warnings

In [2]:
warnings.filterwarnings('ignore')

# csv -> Format 1

In [3]:
start_time = "2018-08-01 00:00:00"
end_time = "2018-08-08 00:00:00"

In [4]:
format1_path = '../data/data_format1_201808.h5' #"D:/Dropbox/MAFS-2018/data/data_format1_20180801_20180807.h5"
format2_path = '../data/data_format2_201808.h5' #"D:/Dropbox/MAFS-2018/data/data_format2_20180801_20180807.h5"

In [5]:
h5data = pd.HDFStore(format1_path)

In [6]:
file_list = glob.glob('D:/Dropbox/MAFS-2018/data/*.csv')

In [7]:
for file in file_list:
    symbol = file.split(sep="\\")[-1].split(sep="_")[0]
    df = pd.read_csv(file, parse_dates=["time"])
    df = df[(df.time >= start_time) & (df.time < end_time)]
    df.set_index(keys="time", drop=True, inplace=True)
    h5data[symbol] = df
    print(symbol)

BCH-USD
BTC-USD
ETH-USD
LTC-USD


In [11]:
h5data.close()

# Format 1 -> Format 2

In [9]:
import h5py

In [10]:
def read_h5(path, dtype='dataframe'):
    assert dtype in ['dataframe', 'nparray'], "You didn't get the right dtype"
    dataList = list()
    keys = list()
    
    f = h5py.File(path, 'r')
    for key in f.keys():
        keys.append(key)
    f.close()
    
    for key in keys:
        if dtype=='dataframe':
            dataList.append(pd.read_hdf(path, key))
        else:
            dataList.append(pd.read_hdf(path, key).values)
                        
    return dataList, keys

In [11]:
(l, keys) = read_h5(format1_path)

In [12]:
index = l[0].index
h5f = h5py.File(format2_path, 'w')
for i in index:
    min_data = l[0].loc[[i]]
    for j in range(1, len(l)):
        min_data = min_data.append(l[j].loc[[i]])
    h5f.create_dataset(str(i), data=min_data.values)

In [13]:
h5f.close()